In [86]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import VGG16
from keras.preprocessing import image
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2

In [87]:
from keras.preprocessing.image import ImageDataGenerator


# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
validation_datagen = ImageDataGenerator(rescale = 1.0/255. )

In [88]:
train_dir = '../Proyecto Mlearning/DataSet/Training'  # Directorio 

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    classes=['Limones', 'Mandarina', 'Pomelos', 'Naranjas'],  # Especificar las carpeta a usar
    class_mode='categorical'
)


Found 2432 images belonging to 4 classes.


Usar el modelo VGG16 como base y añadir capas:

In [89]:

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(4, activation='softmax')
])


In [90]:
base_model


Entrena el modelo con tus datos:

In [92]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, epochs=5)


Epoch 1/5
27/76 [=========>....................] - ETA: 5:16 - loss: 5.7951 - accuracy: 0.5521

KeyboardInterrupt: 

Evaluación del Modelo

In [72]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_3 (Flatten)         (None, 25088)             0         
                                                                 
 dense_6 (Dense)             (None, 1024)              25691136  
                                                                 
 dense_7 (Dense)             (None, 4)                 4100      
                                                                 
Total params: 40409924 (154.15 MB)
Trainable params: 25695236 (98.02 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


modelo entrenado para predecir nuevas imágenes

In [46]:


def predict_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    
    prediction = model.predict(img_array)
    return np.argmax(prediction, axis=1)

# Ejemplo de uso
predict_image('../Proyecto Mlearning/DataSet/Test/Limon/3_100.jpg')


1/1 [==============================] - 0s 332ms/step


array([0])

In [77]:
class_names = {0: "Limon", 1: "Mandarina", 2: "Naranja", 3: "Pomelo"}  # Continúa para todas las clases que tengas

In [71]:
def predict_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    
    prediction = model.predict(img_array)
    class_index = np.argmax(prediction, axis=1)
    return class_names[class_index[0]]  # Usa el índice para obtener el nombre de la clase

# Ejemplo de uso
fruit_name = predict_image('../Proyecto Mlearning/DataSet/Test/Mandarina/r_300_100.jpg')
print(fruit_name)

1/1 [==============================] - 0s 274ms/step
Limon
